In [1]:
import pyopencl as cl #ONLY RUN THIS IF YOU'RE USING GPU

In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\18199\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\18199\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\18199\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

In [4]:
pre_df = pd.read_csv('./data/pre_soccer_extraction.csv')
post_df = pd.read_csv('./data/post_soccer_extraction.csv')

In [5]:
pre_df

,ptitle,pscore,pid,pbody,pcreated,comment,cauthor,ccreated
0,[Pre-Match Thread] Porto vs Juventus (Champion...,60,lls3ua,**Porto vs Juventus**\n\nCompetition: UEFA Cha...,1.613589e+09,If you can’t stop Sporting from winning the le...,idek0k,1.613592e+09
1,[Pre-Match Thread] Porto vs Juventus (Champion...,60,lls3ua,**Porto vs Juventus**\n\nCompetition: UEFA Cha...,1.613589e+09,"I don't follow Liga NOS closely, so any Portug...",kuzjaruge,1.613590e+09
2,[Pre-Match Thread] Porto vs Juventus (Champion...,60,lls3ua,**Porto vs Juventus**\n\nCompetition: UEFA Cha...,1.613589e+09,"Go on Porto, get that win so I can look at the...",TweakyWatson,1.613591e+09
3,[Pre-Match Thread] Porto vs Juventus (Champion...,60,lls3ua,**Porto vs Juventus**\n\nCompetition: UEFA Cha...,1.613589e+09,It’s a shame that fans aren’t allowed in stadi...,rabbitvinyl,1.613596e+09
4,[Pre-Match Thread] Porto vs Juventus (Champion...,60,lls3ua,**Porto vs Juventus**\n\nCompetition: UEFA Cha...,1.613589e+09,"Juventus will take this easily ,porto is a fai...",NunoxGames,1.613591e+09
...,...,...,...,...,...,...,...,...
18322,Pre Match Thread: Real Madrid vs Ajax [UCL] (2...,102,axleo6,#Real Madrid [](#sprite1-p9) - [](#sprite1-p22...,1.551823e+09,>Implying Liverpool won't get stomped in München,KDB_Citizen,1.551841e+09
18323,Pre Match Thread: Real Madrid vs Ajax [UCL] (2...,102,axleo6,#Real Madrid [](#sprite1-p9) - [](#sprite1-p22...,1.551823e+09,"I said it in the summer and I'll say it again,...",OleoleCholoSimeone,1.551828e+09
18324,Pre Match Thread: Real Madrid vs Ajax [UCL] (2...,102,axleo6,#Real Madrid [](#sprite1-p9) - [](#sprite1-p22...,1.551823e+09,I LOVE DIS GAMEEEEEEEEEEEEEEEEEE,YerCatalanPadre,1.551850e+09
18325,Pre Match Thread: Real Madrid vs Ajax [UCL] (2...,102,axleo6,#Real Madrid [](#sprite1-p9) - [](#sprite1-p22...,1.551823e+09,Doesn't make it true even if you say it thrice,Ajk320,1.551836e+09


In [6]:
post_df

,ptitle,pscore,pid,pbody,pcreated,comment,cauthor,ccreated
0,Post Match Thread: FC Barcelona 1-4 Paris Sain...,2435,llen1t,#[](#sprite1-p6) vs [](#sprite1-p35)\n\n**Venu...,1.613541e+09,"**GIFs / Stats / Quotes** *I am a bot, and th...",AutoModerator,1.613541e+09
1,Post Match Thread: FC Barcelona 1-4 Paris Sain...,2435,llen1t,#[](#sprite1-p6) vs [](#sprite1-p35)\n\n**Venu...,1.613541e+09,[Mbappe v Barca](https://preview.redd.it/ky8lk...,CreateNewAccountsss,1.613541e+09
2,Post Match Thread: FC Barcelona 1-4 Paris Sain...,2435,llen1t,#[](#sprite1-p6) vs [](#sprite1-p35)\n\n**Venu...,1.613541e+09,Good guy PSG saved Barcelona the embarrassment...,codespyder,1.613541e+09
3,Post Match Thread: FC Barcelona 1-4 Paris Sain...,2435,llen1t,#[](#sprite1-p6) vs [](#sprite1-p35)\n\n**Venu...,1.613541e+09,"On the bright side for Barcelona, this is a 50...",TheChiffre,1.613541e+09
4,Post Match Thread: FC Barcelona 1-4 Paris Sain...,2435,llen1t,#[](#sprite1-p6) vs [](#sprite1-p35)\n\n**Venu...,1.613541e+09,"Koeman set it up perfectly for Remontada 2.0, ...",SomeIrishFiend,1.613541e+09
...,...,...,...,...,...,...,...,...
36374,Post-Match Thread: Arsenal 3-0 Newcastle Unite...,445,l04pi5,# [](#sprite1-p1) **Arsenal** [3 - 0](#bar-3-w...,1.611035e+09,They overpaid for Joelinton but he isn't that ...,speedycar1,1.611068e+09
36375,Post-Match Thread: Arsenal 3-0 Newcastle Unite...,445,l04pi5,# [](#sprite1-p1) **Arsenal** [3 - 0](#bar-3-w...,1.611035e+09,Ah i wasn't aware. English football punditry j...,stevefingmadden,1.611100e+09
36376,Post-Match Thread: Arsenal 3-0 Newcastle Unite...,445,l04pi5,# [](#sprite1-p1) **Arsenal** [3 - 0](#bar-3-w...,1.611035e+09,Got to wonder what Wilson must be thinking rig...,hydraulicson,1.611037e+09
36377,Post-Match Thread: Arsenal 3-0 Newcastle Unite...,445,l04pi5,# [](#sprite1-p1) **Arsenal** [3 - 0](#bar-3-w...,1.611035e+09,I wouldn't blame him at all if he was regretti...,King_Hobbes,1.611037e+09


In [7]:
def TokenizeProcess(df):
    word_lemmatizer = WordNetLemmatizer()

    df_tk = pd.DataFrame(columns=['ptitle', 'pscore', 'pid', 'pbody', 'pcreated', 'comment', 'cauthor', 'ccreated'])

    for (ptitle, pscore, pid, pbody, pcreated, comment, cauthor, ccreated) in df.values.tolist():
        # Tokenize
        tokens = word_tokenize(str(comment))

        # Strip punctuation
        punctuation_list = str.maketrans('', '', string.punctuation)
        tokens_strp = [w.translate(punctuation_list) for w in tokens]

        # Remove other non-alphabetic tokens
        words = [word for word in tokens_strp if word.isalpha()]

        # Stop words
        stop_words = set(stopwords.words('english'))
        words = [w for w in words if not w in stop_words]

        # Lemmatize
        # Note: We might need to do this for the brands we want to detect as well
        words_lm = [word_lemmatizer.lemmatize(w) for w in words]
        df_tk = df_tk.append({"ptitle": ptitle, 'pscore':pscore, 'pid':pid, 'pbody':pbody, 'pcreated': pcreated, 'comment': words_lm, 'cauthor': cauthor} ,ignore_index=True) 
    return df_tk

In [8]:
pre_df_tk = TokenizeProcess(pre_df)

In [9]:
post_df_tk = TokenizeProcess(post_df)

In [10]:
pre_df_tk.head(10)

,ptitle,pscore,pid,pbody,pcreated,comment,cauthor,ccreated
0,[Pre-Match Thread] Porto vs Juventus (Champion...,60,lls3ua,**Porto vs Juventus**\n\nCompetition: UEFA Cha...,1.613589e+09,"[If, stop, Sporting, winning, league, least, m...",idek0k,NaN
1,[Pre-Match Thread] Porto vs Juventus (Champion...,60,lls3ua,**Porto vs Juventus**\n\nCompetition: UEFA Cha...,1.613589e+09,"[I, nt, follow, Liga, NOS, closely, Portuguese...",kuzjaruge,NaN
2,[Pre-Match Thread] Porto vs Juventus (Champion...,60,lls3ua,**Porto vs Juventus**\n\nCompetition: UEFA Cha...,1.613589e+09,"[Go, Porto, get, win, I, look, match, thread, ...",TweakyWatson,NaN
3,[Pre-Match Thread] Porto vs Juventus (Champion...,60,lls3ua,**Porto vs Juventus**\n\nCompetition: UEFA Cha...,1.613589e+09,"[It, shame, fan, allowed, stadium, I, think, w...",rabbitvinyl,NaN
4,[Pre-Match Thread] Porto vs Juventus (Champion...,60,lls3ua,**Porto vs Juventus**\n\nCompetition: UEFA Cha...,1.613589e+09,"[Juventus, take, easily, porto, fairly, decent...",NunoxGames,NaN
5,[Pre-Match Thread] Porto vs Juventus (Champion...,60,lls3ua,**Porto vs Juventus**\n\nCompetition: UEFA Cha...,1.613589e+09,"[Oh, Cuadrado, injured, Sucks, I, looking, for...",atropicalpenguin,NaN
6,[Pre-Match Thread] Porto vs Juventus (Champion...,60,lls3ua,**Porto vs Juventus**\n\nCompetition: UEFA Cha...,1.613589e+09,"[Marega, masterclass]",Homygod319,NaN
7,[Pre-Match Thread] Porto vs Juventus (Champion...,60,lls3ua,**Porto vs Juventus**\n\nCompetition: UEFA Cha...,1.613589e+09,"[McKennie, Bentancur, Rabiot, starting, might,...",jv17fortheW,NaN
8,[Pre-Match Thread] Porto vs Juventus (Champion...,60,lls3ua,**Porto vs Juventus**\n\nCompetition: UEFA Cha...,1.613589e+09,"[I, fucking, hope, Bernardeschi, nt, start, du...",Handyman2116,NaN
9,[Pre-Match Thread] Porto vs Juventus (Champion...,60,lls3ua,**Porto vs Juventus**\n\nCompetition: UEFA Cha...,1.613589e+09,"[happy, get, watch, goat, play, champion, leag...",90degreeturnsbtw,NaN


In [11]:
post_df.head(10)

,ptitle,pscore,pid,pbody,pcreated,comment,cauthor,ccreated
0,Post Match Thread: FC Barcelona 1-4 Paris Sain...,2435,llen1t,#[](#sprite1-p6) vs [](#sprite1-p35)\n\n**Venu...,1.613541e+09,"**GIFs / Stats / Quotes** *I am a bot, and th...",AutoModerator,1.613541e+09
1,Post Match Thread: FC Barcelona 1-4 Paris Sain...,2435,llen1t,#[](#sprite1-p6) vs [](#sprite1-p35)\n\n**Venu...,1.613541e+09,[Mbappe v Barca](https://preview.redd.it/ky8lk...,CreateNewAccountsss,1.613541e+09
2,Post Match Thread: FC Barcelona 1-4 Paris Sain...,2435,llen1t,#[](#sprite1-p6) vs [](#sprite1-p35)\n\n**Venu...,1.613541e+09,Good guy PSG saved Barcelona the embarrassment...,codespyder,1.613541e+09
3,Post Match Thread: FC Barcelona 1-4 Paris Sain...,2435,llen1t,#[](#sprite1-p6) vs [](#sprite1-p35)\n\n**Venu...,1.613541e+09,"On the bright side for Barcelona, this is a 50...",TheChiffre,1.613541e+09
4,Post Match Thread: FC Barcelona 1-4 Paris Sain...,2435,llen1t,#[](#sprite1-p6) vs [](#sprite1-p35)\n\n**Venu...,1.613541e+09,"Koeman set it up perfectly for Remontada 2.0, ...",SomeIrishFiend,1.613541e+09
5,Post Match Thread: FC Barcelona 1-4 Paris Sain...,2435,llen1t,#[](#sprite1-p6) vs [](#sprite1-p35)\n\n**Venu...,1.613541e+09,A complete list of every time a team has come ...,gnorrn,1.613541e+09
6,Post Match Thread: FC Barcelona 1-4 Paris Sain...,2435,llen1t,#[](#sprite1-p6) vs [](#sprite1-p35)\n\n**Venu...,1.613541e+09,I can't remember a Barcelona season like this....,MrSteelCity,1.613541e+09
7,Post Match Thread: FC Barcelona 1-4 Paris Sain...,2435,llen1t,#[](#sprite1-p6) vs [](#sprite1-p35)\n\n**Venu...,1.613541e+09,It's a bad day to have eyes.,razycal970,1.613541e+09
8,Post Match Thread: FC Barcelona 1-4 Paris Sain...,2435,llen1t,#[](#sprite1-p6) vs [](#sprite1-p35)\n\n**Venu...,1.613541e+09,PSG having a comfortable lead after the first ...,Sdub4,1.613541e+09
9,Post Match Thread: FC Barcelona 1-4 Paris Sain...,2435,llen1t,#[](#sprite1-p6) vs [](#sprite1-p35)\n\n**Venu...,1.613541e+09,Can’t wait to this to be the night which convi...,LosTerminators,1.613541e+09


In [13]:
pre_df_tk.to_csv("./data/pre_soccer_tokenized.csv",index=False)
post_df_tk.to_csv("./data/post_soccer_tokenized.csv",index=False)